In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END


In [2]:
load_dotenv()

model = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.1-8b-instant"
)

In [3]:
#Creating State

class blog_state(TypedDict):

    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: blog_state) -> blog_state:

    #fetch title
    title = state['title']

    #Call LLM generate outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    #Update state
    state['outline'] = outline

    return state

In [5]:
def create_blog(state: blog_state) -> blog_state:

    #Fetch Outline
    title = state['title']
    outline = state['outline']

    #Call LLM generate blog
    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline}'

    content = model.invoke(prompt).content

    #Update state
    state[content] = content

    return state

In [7]:
graph = StateGraph(blog_state)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)


graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [8]:
initial_state = {'title': "rise of AI in world"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'rise of AI in world', 'outline': "**Title:** The Rise of AI in the World: Transforming Industries and Revolutionizing Lives\n\n**I. Introduction**\n\n- Brief overview of the increasing importance of AI in modern society\n- Importance of understanding the rise of AI and its impact on various industries\n- Thesis statement: The rise of AI is transforming the world, revolutionizing industries, and changing lives, and its impact will only continue to grow in the coming years.\n\n**II. History of AI: From its Humble Beginnings to the Present Day**\n\n- Overview of the early days of AI, including the development of the first AI programs and the Dartmouth Summer Research Project on Artificial Intelligence in 1956\n- Discussion of the key milestones in AI history, including the development of machine learning, natural language processing, and computer vision\n- Explanation of how AI has evolved over time and how it has become more sophisticated and integrated into various industries

In [9]:
print(final_state['outline'])

**Title:** The Rise of AI in the World: Transforming Industries and Revolutionizing Lives

**I. Introduction**

- Brief overview of the increasing importance of AI in modern society
- Importance of understanding the rise of AI and its impact on various industries
- Thesis statement: The rise of AI is transforming the world, revolutionizing industries, and changing lives, and its impact will only continue to grow in the coming years.

**II. History of AI: From its Humble Beginnings to the Present Day**

- Overview of the early days of AI, including the development of the first AI programs and the Dartmouth Summer Research Project on Artificial Intelligence in 1956
- Discussion of the key milestones in AI history, including the development of machine learning, natural language processing, and computer vision
- Explanation of how AI has evolved over time and how it has become more sophisticated and integrated into various industries

**III. The Current State of AI: Applications and Indust